**Aygaz Görüntü İşleme Bootcamp Projesi Proje Amacı**

Bu projede, görüntü işleme ve derin öğrenme teknikleri kullanılarak 10 farklı hayvan türünün sınıflandırılması hedeflenmiştir. Modelin dayanıklılığı, manipüle edilmiş test setleri ve renk sabitliği algoritmalarıyla test edilmiştir.

**1. Veri Setini Kullanma ve Dinamik Toplama**

Veri seti, Animals with Attributes 2 (AwA2) klasör yapısına sahiptir.
Her alt klasör bir sınıfı temsil eder.
Dinamik bir yapı ile tüm görseller ve sınıf etiketleri toplanır.


In [ ]:
import os

# Veri setinin ana klasörü
DATASET_DIR = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"


# Projede kullanılacak sınıflar
selected_classes = ["collie", "dolphin", "elephant", "fox", "moose", 
                    "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]

# Mevcut sınıfları kontrol et
available_classes = [cls for cls in os.listdir(DATASET_DIR) if os.path.isdir(os.path.join(DATASET_DIR, cls))]

print(f"Veri setinde toplam {len(available_classes)} sınıf bulunuyor.")
print(f"Mevcut sınıflar: {available_classes}")

# Seçilen sınıfların mevcut olup olmadığını doğrula
missing_classes = [cls for cls in selected_classes if cls not in available_classes]
if missing_classes:
    print(f"Uyarı: Aşağıdaki sınıflar veri setinde bulunamadı: {missing_classes}")
else:
    print(f"Tüm seçilen sınıflar mevcut: {selected_classes}")




**2. Görsellerin Yüklenmesi ve İşlenmesi**

* Görseller, 128x128 piksel boyutuna küçültülür.
* Görseller 0-1 aralığında normalize edilir.
* Her sınıftan maksimum 650 görsel alınır.

In [ ]:
import os
import cv2
import numpy as np

# 1. Veri Seti ve Sınıf Parametreleri
DATASET_DIR ="/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"
selected_classes = ["collie", "dolphin", "elephant", "fox", "moose", 
                    "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]

IMAGE_SIZE = (128, 128)
MAX_IMAGES_PER_CLASS = 650

def prepare_images(dataset_dir, selected_classes, image_size, max_images_per_class):
    images, labels = [], []
    for class_name in selected_classes:
        class_path = os.path.join(dataset_dir, class_name)
        image_files = os.listdir(class_path)[:max_images_per_class]
        for img_name in image_files:
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img_resized = cv2.resize(img, image_size)
                images.append(img_resized / 255.0)  # Normalizasyon
                labels.append(class_name)
    return np.array(images), np.array(labels)

X, y = prepare_images(DATASET_DIR, selected_classes, IMAGE_SIZE, MAX_IMAGES_PER_CLASS)
print(f"Toplam Görsel: {X.shape[0]}, Sınıflar: {np.unique(y)}")


**3. Eğitim ve Test Setinin Ayrılması**

* Veriler %70 eğitim ve %30 test olarak ayrılmıştır.
* Etiketler, LabelEncoder ile sayısal değerlere çevrilmiş ve ardından one-hot encoding uygulanmıştır.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Etiketleri dijital değerlere dönüştürme
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)  # y, 2. adımda tanımlanan etiketlerdir

# One-hot encoding
y_categorical = to_categorical(y_encoded)

# Eğitim ve test setine ayırma (%70 eğitim, %30 test)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.3, random_state=42)

# Çıktı boyutlarını yazdırma
print(f"Eğitim veri boyutu: {X_train.shape}, Test veri boyutu: {X_test.shape}")



**4. Veri Artırımı (Augmentation)**

* Eğitim verisi üzerinde veri artırma yöntemleri uygulanmıştır.

Aşağıdaki işlemler gerçekleştirilmiştir:

Döndürme (20 dereceye kadar)

Yakınlaştırma/Uzaklaştırma (%20)

Yatay eksende ayna yansıması

Yatay ve dikey kaydırma

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.2
)
datagen.fit(X_train)


**5. CNN Modelinin Tasarlanması**

Modelde kullanılan katmanlar:
* Conv2D: Görüntüden özellik çıkarma.
* MaxPooling2D: Önemli bilgileri seçme.
* Flatten: Matrisleri vektör haline getirme.
* Dense: Tam bağlantılı katmanlar.
* Dropout: Ezberlemeyi önleme.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

model = Sequential([
    Input(shape=(128, 128, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()




**6. Modelin Eğitilmesi ve Test Edilmesi**

* Model 30 epoch boyunca eğitilmiştir.
* Eğitim ve doğrulama setindeki doğruluk oranları hesaplanmıştır.



In [ ]:
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=30
)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test doğruluğu: {test_acc * 100:.2f}%")



**7. Manipüle Edilmiş Görüntülerle Test**

Görseller manipüle edilmiştir:

Kontrast artırma.

Saat yönünde döndürme.

Modelin dayanıklılığı ölçülmüştür.

In [ ]:
def manipulate_images(images):
    manipulated_images = []
    for img in images:
        bright = cv2.convertScaleAbs(img, alpha=1.5, beta=20)  # Kontrast artırma
        rotated = cv2.rotate(bright, cv2.ROTATE_90_CLOCKWISE)
        manipulated_images.append(rotated)
    return np.array(manipulated_images)

X_test_manipulated = manipulate_images(X_test)
manipulated_loss, manipulated_acc = model.evaluate(X_test_manipulated, y_test)
print(f"Manipüle edilmiş doğruluk: {manipulated_acc * 100:.2f}%")


**8. Renk Sabitliği Algoritması**

Görsellere renk sabitliği algoritması uygulanarak normalize edilmiştir.

"Gray World Assumption" yöntemi kullanılmıştır

In [ ]:
def apply_color_constancy(images):
    processed_images = []
    for img in images:
        avg_channels = np.mean(img, axis=(0, 1))
        avg_intensity = np.mean(avg_channels)
        scale = avg_intensity / avg_channels
        balanced = img * scale.reshape(1, 1, -1)
        processed_images.append(np.clip(balanced, 0, 1))
    return np.array(processed_images)

X_test_color_const = apply_color_constancy(X_test)
color_loss, color_acc = model.evaluate(X_test_color_const, y_test)
print(f"Renk sabitliği doğruluğu: {color_acc * 100:.2f}%")


**9. Sonuçların Karşılaştırılması ve Raporlama**

Orijinal, manipüle edilmiş ve renk sabitliği uygulanmış görüntülerin doğruluk oranları görselleştirilmiştir.

In [ ]:
import matplotlib.pyplot as plt

accuracies = [test_acc * 100, manipulated_acc * 100, color_acc * 100]
labels = ['Orijinal', 'Manipüle', 'Renk Sabitliği']
plt.bar(labels, accuracies, color=['blue', 'orange', 'green'])
plt.ylabel('Doğruluk (%)')
plt.title('Performans Karşılaştırması')
plt.show()


In [ ]:
# Test sonuçlarının karşılaştırılması ve çıktılar
print("Sonuçlar:")

# Orijinal test seti doğruluğunu yazdır
print(f"Orijinal Test Seti Doğruluğu: {test_acc * 100:.2f}%")

# Manipüle edilmiş test seti doğruluğunu yazdır
print(f"Manipüle Edilmiş Test Seti Doğruluğu: {manipulated_acc * 100:.2f}%")

# Renk sabitliği uygulanmış test seti doğruluğunu yazdır
print(f"Renk Sabitliği Uygulanmış Test Seti Doğruluğu: {color_acc * 100:.2f}%")


**Sonuç ve Çözüm Önerileri**
* Manipüle Edilmiş Görseller: Modelin manipüle edilmiş görsellerde performansı oldukça düşüktür. Eğitim sürecine manipüle edilmiş veri dahil edilerek bu performans artırılabilir.
* Renk Sabitliği: Renk sabitliği algoritması, manipüle edilmiş görseller üzerinde modelin doğruluğunu önemli ölçüde artırmıştır.
* Sonuç: Model, orijinal test setinde %67.64 doğrulukla temel bir başarı sergilemiştir. Performansı artırmak için daha karmaşık model mimarileri veya veri artırma teknikleri kullanılabilir.